In [ ]:
import nibabel as nib
import numpy as np
import os

all_data = []
flat_data = []
ages = []
sexes = []
non_zeros = np.load('non_zeros.npy')
fails = 0
rootDir = 'data'
for dirName, subdirList, fileList in os.walk(rootDir):
    if 'T88' in dirName and 't4_files' not in dirName: 
        ds = dirName.split('/')
        sessionId = ds[2]
        metadataFileName = '/'.join(ds[0:3]) + '/' + sessionId + '.txt'
        dataFileName = '/'.join(ds) + '/' + sessionId + '_mpr_n4_anon_111_t88_masked_gfc.img'
        try:
            img = nib.load(dataFileName)
        except FileNotFoundError:
            dataFileName = '/'.join(ds) + '/' + sessionId + '_mpr_n3_anon_111_t88_masked_gfc.img'
        try:
            img = nib.load(dataFileName)
        except FileNotFoundError:
            fails += 1
            continue
        data = img.get_data()
        # all_data.append(data)
        with open(metadataFileName) as f:
            for line in f:
                if "AGE:" in line:
                    ages.append(list(filter(None, line.split()))[1])
                elif "M/F:" in line:
                    sexes.append(list(filter(None, line.split()))[1])
                    break
        
        # creating the non_zero array of indexes 
        # inds = np.where(data.flatten() > 0)
        # if non_zeros.size == 0: 
        #    non_zeros = inds[0]
        # else: 
        #    non_zeros = np.intersect1d(non_zeros, inds[0])
        
        flat_data.append(data.flatten()[non_zeros])
        
        
# all_data = np.asarray(all_data, float)
flat_data_np = np.array(flat_data)

# WARNING: UNCOMMENTING THE FOLLOWING LINES WILL OVERWRITE YOUR BACKUPS

# np.save('flat_data', flat_data_np)
# np.save('ages', ages)
# np.save('sexes', sexes)

In [ ]:
import numpy as np

ages = np.load('ages.npy')
#sexes = np.load('sexes.npy')
flat_data_np = np.load('flat_data.npy')

In [ ]:
# finding items, that are equal everywhere
notTheSame = np.where(np.not_equal(flat_data_np[0], flat_data_np[1]))[0]
for element in flat_data_np:
    inds = np.where(np.not_equal(flat_data_np[0], flat_data_np[1]))[0]
    notTheSame = np.intersect1d(notTheSame, inds)
    
flat_data_filtered = flat_data_np[:,notTheSame]
flat_data_np = None
notTheSame = None # freeing up some memory
flat_data_filtered.shape

In [ ]:
from sklearn.decomposition import IncrementalPCA

pca = IncrementalPCA(batch_size=105, n_components=40)
model = pca.fit(flat_data_filtered, ages)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(model.explained_variance_)

In [ ]:
transformedData = model.transform(flat_data_filtered, ages)
#np.save('transformedData',transformedData)

In [ ]:
import numpy as np

transformedData = np.load('transformedData.npy')
ages = np.load('ages.npy')
sexes = np.load('sexes.npy')

In [ ]:
# from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(transformedData, ages, test_size=0.2)

mod = LogisticRegression(n_jobs=-1)
mod.fit(X_train, y_train)
#y_out = np.array(mod.predict(X_test))
# y_test = np.array(y_test, dtype='float64')
score = mean_squared_error(np.array(y_test, dtype='float64'), np.array(mod.predict(X_test),dtype='float64'))

for i in range(0,y_test.shape[0]):
    print mod.predict(X_test[i]), y_test[i]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import KFold,cross_val_score

# define base mode
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(transformedData.shape[1], input_dim=transformedData.shape[1], init='normal', activation='sigmoid'))
    #model.add(Dense(20, init='normal', activation='relu'))
    #model.add(Dense(25, init='normal', activation='relu'))
    #model.add(Dense(10, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model


#steps = []
#steps.append(('scale',StandardScaler()))
#steps.append(('neural_net', KerasRegressor(build_fn=baseline_model, nb_epoch=300, batch_size=50, verbose=0)))

# fix random seed for reproducibility
#seed = 13
#np.random.seed(seed)
#kfold = KFold(transformedData.shape[0],n_folds=8, random_state=seed)
#results = cross_val_score(Pipeline(steps), transformedData, ages, cv=kfold)
#print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

import matplotlib.pyplot as plt
%matplotlib inline

scaler = StandardScaler().fit(transformedData)
X = scaler.transform(transformedData)
model = KerasRegressor(build_fn=baseline_model, nb_epoch=5000, batch_size=50, verbose=0)
history = model.fit(X[sexes!='Male'], ages[sexes!='Male'], validation_split=0.15, verbose=0)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

after PCA with 25 dim:
1 layer: 166.48 (45.64) MSE
2 layers: 262.73 (90.85) MSE
more is even worse



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(transformedData[sexes!='Male'], ages[sexes!='Male'], test_size=0.2)

for i in range(0,y_test.shape[0]):
    #print mod.predict(X_test[i].reshape(1,25)), y_test[i]
    pass
print sum(sexes=='Male'), sum(sexes!='Male')
plt.hist(np.array(ages, dtype='int16'))

In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))